# HTTP Requests
---
This notebook introduces the `requests` library, which we can use to retrieve data from the web.

Some data resources are provided online as **APIs** (Application Programming Interfaces), which can be accessed using a protocol called HTTP (HyperText Transfer Protocol). These interfaces may allow us to retrieve information in various ways, for example by asking for a particular record or making a search.

In this notebook we will look at an API provided by the [US Geological Survey](https://www.usgs.gov/programs/earthquake-hazards) providing continually-updated information about earthquakes around the world.

Firstly, take a look at the web form for searching manually. Go to https://earthquake.usgs.gov/earthquakes/search/

Notice all of the different search options and try some searches out.

The results are presented as a list and an interactive map.

<img src='../resources/earthquakes.png'>

---
## Making a request
But what if we want to retrieve the data themselves? Here is an example using `requests`:

In [4]:
import requests

# An endpoint is a URL that accepts requests
endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

# The parameters for our search, in the form of a python dictionary
ps = { "format": "csv", 
       "starttime": "2024-09-27", 
       "endtime": "2024-09-28",
       "minmagnitude": 4.5  
     }

response = requests.get(endpoint, params=ps)

print(response.text)


time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
2024-09-27T09:37:26.438Z,-13.6744,-171.3992,10,4.5,mb,17,91,0.439,0.65,us,us7000ngsh,2024-09-27T10:30:12.040Z,"30 km NE of Lufilufi, Samoa",earthquake,1.97,1.902,0.222,7,reviewed,us,us
2024-09-27T05:40:55.288Z,-7.1974,147.3316,58.397,4.9,mb,45,81,2.2,1.01,us,us7000ngr9,2024-09-27T06:02:07.040Z,"64 km SE of Lae, Papua New Guinea",earthquake,8.74,7.522,0.076,55,reviewed,us,us
2024-09-27T04:36:20.269Z,8.9423,40.0832,10,4.9,mb,47,195,3.743,0.67,us,us7000ngr4,2024-09-27T10:32:18.040Z,"10 km WSW of Āwash, Ethiopia",earthquake,13.55,1.957,0.077,53,reviewed,us,us
2024-09-27T03:39:31.854Z,11.8785,-87.6523,51.404,5.1,mww,70,135,1.021,0.93,us,us7000ngr0,2024-09-27T03:58:30.040Z,"84 km SW of Corinto, Nicaragua",earthquake,6.64,7.701,0.08,15,reviewed,us,us
2024-09-27T02:23:00.695Z,-10.9539,161.4818,10,5.3,mww,37,101,2.133,1.14,us,us7000ngqn,

An *endpoint* is a particular URL (web address) that will accept a request in a particular format and return a *response*. 

The USGS endpoint is able to return data in CSV format, which we can access using the `.text` attribute.

The parameter options for this endpoint are explained 
[here](https://earthquake.usgs.gov/fdsnws/event/1/?ref=springboard).

---
## Converting to a DataFrame
If we want to create a DataFrame from this CSV text, we can do the following:

In [5]:
import pandas as pd
import io

# make a string buffer from the CSV text
buf = io.StringIO(response.text) 

# pandas can now read the data from the buffer
data = pd.read_csv(buf)          

data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-09-27T09:37:26.438Z,-13.6744,-171.3992,10.000,4.5,mb,17,91,0.439,0.65,...,2024-09-27T10:30:12.040Z,"30 km NE of Lufilufi, Samoa",earthquake,1.97,1.902,0.222,7,reviewed,us,us
1,2024-09-27T05:40:55.288Z,-7.1974,147.3316,58.397,4.9,mb,45,81,2.200,1.01,...,2024-09-27T06:02:07.040Z,"64 km SE of Lae, Papua New Guinea",earthquake,8.74,7.522,0.076,55,reviewed,us,us
2,2024-09-27T04:36:20.269Z,8.9423,40.0832,10.000,4.9,mb,47,195,3.743,0.67,...,2024-09-27T10:32:18.040Z,"10 km WSW of Āwash, Ethiopia",earthquake,13.55,1.957,0.077,53,reviewed,us,us
3,2024-09-27T03:39:31.854Z,11.8785,-87.6523,51.404,5.1,mww,70,135,1.021,0.93,...,2024-09-27T03:58:30.040Z,"84 km SW of Corinto, Nicaragua",earthquake,6.64,7.701,0.080,15,reviewed,us,us
4,2024-09-27T02:23:00.695Z,-10.9539,161.4818,10.000,5.3,mww,37,101,2.133,1.14,...,2024-09-27T02:41:10.040Z,"73 km SW of Kirakira, Solomon Islands",earthquake,7.68,1.785,0.098,10,reviewed,us,us


---
### JSON

We are getting comfortable with CSV format, but there are many other data formats that are in common use.

When working with APIs, we often encounter the **JSON** (JavaScript Object Notation) format. 

The USGS endpoint can return responses in [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), a particular flavour of JSON which is used to describe geolocated objects.

In [6]:
endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

ps = { "format": "geojson", 
       "starttime": "2023-09-27", 
       "endtime": "2023-09-28",
       "minmagnitude": 4.8  
     }

response = requests.get(endpoint, params=ps)

print(response.text)


{"type":"FeatureCollection","metadata":{"generated":1727434853000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-09-27&endtime=2023-09-28&minmagnitude=4.8","title":"USGS Earthquakes","status":200,"api":"1.14.1","count":5},"features":[{"type":"Feature","properties":{"mag":4.8,"place":"296 km ESE of Levuka, Fiji","time":1695837640270,"updated":1702159240040,"tz":null,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/us6000lb63","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000lb63&format=geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"reviewed","tsunami":0,"sig":354,"net":"us","code":"6000lb63","ids":",us6000lb63,","sources":",us,","types":",origin,phase-data,","nst":104,"dmin":3.475,"rms":0.81,"gap":44,"magType":"mb","type":"earthquake","title":"M 4.8 - 296 km ESE of Levuka, Fiji"},"geometry":{"type":"Point","coordinates":[-178.0139,-18.8999,613.904]},"id":"us6000lb63"},
{"type":"Feature","properties

Notice that JSON data consists of nested dictionaries (marked by **{ }**) and lists (marked by **\[ ]**).
It is a very flexible format, and can be used to describe data structures that are more complex than just tables.


If you need to work with JSON data, the built-in `json` module is very handy.

In [7]:
import json
data_dict = json.loads(response.text)
data_dict

{'type': 'FeatureCollection',
 'metadata': {'generated': 1727434853000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-09-27&endtime=2023-09-28&minmagnitude=4.8',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.14.1',
  'count': 5},
 'features': [{'type': 'Feature',
   'properties': {'mag': 4.8,
    'place': '296 km ESE of Levuka, Fiji',
    'time': 1695837640270,
    'updated': 1702159240040,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000lb63',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000lb63&format=geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'reviewed',
    'tsunami': 0,
    'sig': 354,
    'net': 'us',
    'code': '6000lb63',
    'ids': ',us6000lb63,',
    'sources': ',us,',
    'types': ',origin,phase-data,',
    'nst': 104,
    'dmin': 3.475,
    'rms': 0.81,
    'gap': 44,
    'magType': 'mb',
    'type': 

*data_dict* is now just a python dictionary, which we can manipulate as we like.
For example, to print the magnitude of all earthquakes in the response:

In [8]:
for eq in data_dict['features']:
    print(eq['properties']['mag'])

4.8
5.5
5.6
5
5


If needed, we can also convert a JSON dictionary to a pandas DataFrame using [`json_normalize`](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html#pandas.json_normalize):

In [17]:
import pandas as pd

# the 'record_path' argument shows how to locate the table rows within the dict
pd.json_normalize(data_dict, record_path=['features'])

# this doesn't work
# pd.json_normalize(data_dict, record_path=['properties'])
# pd.json_normalize(data_dict)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000lb63,4.8,"296 km ESE of Levuka, Fiji",1695837640270,1702159240040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,None,...,",origin,phase-data,",104,3.475,0.81,44,mb,earthquake,"M 4.8 - 296 km ESE of Levuka, Fiji",Point,"[-178.0139, -18.8999, 613.904]"
1,Feature,us6000lb60,5.5,off the west coast of northern Sumatra,1695836841596,1702159240040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,None,...,",internal-moment-tensor,internal-origin,losspa...",101,4.483,0.68,25,mww,earthquake,M 5.5 - off the west coast of northern Sumatra,Point,"[93.1129, 0.8622, 10]"
2,Feature,us6000lb6a,5.6,Pacific-Antarctic Ridge,1695836255528,1702159240040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,None,...,",losspager,moment-tensor,origin,phase-data,sha...",27,31.771,0.89,120,mww,earthquake,M 5.6 - Pacific-Antarctic Ridge,Point,"[-134.774, -53.5274, 10]"
3,Feature,us6000lb4e,5.0,"6 km W of Amahai, Indonesia",1695831320238,1702159240040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,None,...,",origin,phase-data,",114,1.573,0.63,44,mb,earthquake,"M 5.0 - 6 km W of Amahai, Indonesia",Point,"[128.858, -3.3385, 17.573]"
4,Feature,us6000lb14,5.0,"Izu Islands, Japan region",1695783284172,1702159240040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,None,...,",origin,phase-data,",76,2.354,0.55,119,mb,earthquake,"M 5.0 - Izu Islands, Japan region",Point,"[141.7419, 31.4154, 11.753]"
